In [3]:
import requests
from IPython.display import Image
from IPython.core.display import HTML 

In [1]:
def get_authentication():
    """
    Get authentication from Spotify API using the credentials we have 
    """
    AUTH_URL = 'https://accounts.spotify.com/api/token'
    cid ="0753888cfb694cf98b9332cd343859a3" 
    secret = "0f47a99eb4bc40eea4f3c9d6b3f1df82"
    # POST
    auth_response = requests.post(AUTH_URL, {
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret,
    })

    # convert the response to JSON
    auth_response_data = auth_response.json()
    # save the access token
    access_token = auth_response_data['access_token']

    headers = {
        'Authorization': 'Bearer {token}'.format(token=access_token)}
    return headers 


In [5]:
artistName ='Illenium'
URI = 'https://api.spotify.com/v1/search?query='+ artistName +'&offset=0&limit=1&type=artist'
response = requests.get(url = URI, headers=get_authentication())
response = response.json()
artist_id = response['artists']['items'][0]['id']

In [10]:

album_url = 'https://api.spotify.com/v1/artists/' + artist_id + '/albums' 
response = requests.get(album_url, headers=get_authentication())

In [14]:
t = response.json()
t

{'href': 'https://api.spotify.com/v1/artists/45eNHdiiabvmbp4erw26rg/albums?offset=0&limit=20&include_groups=album,single,compilation,appears_on',
 'items': [{'album_group': 'album',
   'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/45eNHdiiabvmbp4erw26rg'},
     'href': 'https://api.spotify.com/v1/artists/45eNHdiiabvmbp4erw26rg',
     'id': '45eNHdiiabvmbp4erw26rg',
     'name': 'ILLENIUM',
     'type': 'artist',
     'uri': 'spotify:artist:45eNHdiiabvmbp4erw26rg'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CA',
    'CD',
    'CG',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
    'CR',
    'CV',
    'CW',
    'CY',
    'CZ',
    'DE',
    'DJ',
    'DK',
    'DM',
    'DO',
   

In [18]:
t['items'][0]

{'album_group': 'album',
 'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/45eNHdiiabvmbp4erw26rg'},
   'href': 'https://api.spotify.com/v1/artists/45eNHdiiabvmbp4erw26rg',
   'id': '45eNHdiiabvmbp4erw26rg',
   'name': 'ILLENIUM',
   'type': 'artist',
   'uri': 'spotify:artist:45eNHdiiabvmbp4erw26rg'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BY',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',


In [15]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.scdn.co/image/ab67616d0000b273dcf331e276b993d87ed9850b", width = 160)